In [ ]:
# Run this cell to obtain files (e.g., if on colab)
import requests
base_url = "https://raw.githubusercontent.com/shuds13/xstack-benchmark/main/polybench-cuda/results/polybench-cuda-run_2024_09_18_Arg_GCE_node08/"
filenames = ['benchmark_results_clang.csv', 'benchmark_results_gcc.csv', 'run.out']

for filename in filenames:
    url = base_url + filename
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

In [ ]:
%matplotlib inline

import csv
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def plot_from_csv(compiler):
    csv_file = f'benchmark_results_{compiler}.csv'
    benchmarks = []
    tulip_speedups = []
    noelle_speedups = []

    # Read data from CSV
    with open(csv_file, mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            bmark = row['benchmark']
            openmp_time = float(row['openmp_time'])
            tulip_time = float(row['tulip_time']) if row['tulip_time'] else None
            noelle_time = float(row['noelle_time']) if row['noelle_time'] else None

            # Compute speedups
            tulip_speedup = openmp_time / tulip_time if tulip_time else 0
            noelle_speedup = openmp_time / noelle_time if noelle_time else 0

            benchmarks.append(bmark)
            tulip_speedups.append(tulip_speedup)
            noelle_speedups.append(noelle_speedup)

    # Plot the bar chart
    if benchmarks:
        width = 0.35  # the width of the bars
        fig, ax = plt.subplots(figsize=(12, 6))
        x = range(len(benchmarks))

        ax.bar(x, tulip_speedups, width, label=f'tulip.{compiler} speedup')
        ax.bar([p + width for p in x], noelle_speedups, width, label=f'tulip.{compiler}.noelle speedup')

        # Set the y-axis to use integer ticks
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylim(0, max(int(max(tulip_speedups + noelle_speedups, default=0)), 3) + 1)

        # Add gridlines at every integer
        ax.grid(True, which='both', axis='y', linestyle='-', color='gray', linewidth=0.5)
        ax.axhline(1, color='black', linewidth=1.5, label='Speedup = 1')

        ax.set_ylabel('Speedup relative to original OpenMP')
        ax.set_title(f'OpenMP on CPU ({compiler}): Speedup of OpenMP transpiled from CUDA vs. original OpenMP')
        ax.set_xticks([p + width / 2 for p in x])
        ax.set_xticklabels(benchmarks, rotation=70)
        ax.legend()

        plt.tight_layout()
        plt.show()
        plt.close()
    else:
        print('No benchmarks to display.')



In [ ]:
compiler = 'clang' 
plot_from_csv(compiler)


In [ ]:
compiler = 'gcc'
plot_from_csv(compiler)


In [ ]:
# Check results: Show any runs that had an output mismatch to sequential
try:
    with open('run.out', 'r') as file:
        for line in file:
            # Check if the line starts with "Output mismatch"
            if line.startswith('Output mismatch'):
                print(line.strip())
except FileNotFoundError:
    print("The file 'run.out' was not found.")